In [1]:
import matplotlib.pyplot as plt
import numpy as np

image = plt.imread('/Users/yangyongze/Desktop/test.png')  # (1154, 802, 4)
height = image.shape[0]  # rows
width = image.shape[1]  # columns
pixels = height * width
if image.shape[2] == 4:
    image = image[:, :, :3]

unique_colors, counts = np.unique(image.reshape(-1, 3), axis=0, return_counts=True)

print(pixels)
print(len(unique_colors))

print(image[0][0])

925508
11670
[1. 1. 1.]


In [2]:
from pyspark.sql import SparkSession
import os
os.environ['SPARK_HOME'] = 'spark-3.5.1-bin-hadoop3'
os.environ['HADOOP_HOME'] = 'hadoop-3.3.6'
spark = SparkSession.builder.appName("KMeansExample").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/27 22:15:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

image = plt.imread('/Users/yangyongze/Desktop/test.png')  # (1154, 802, 4)
if image.shape[2] == 4:
    image = image[:, :, :3]
image = image.reshape(-1, 3)
data_df = spark.createDataFrame(image.tolist(), ["feature1", "feature2", "feature3"])
assembler = VectorAssembler(inputCols=["feature1", "feature2", "feature3"], outputCol="features")
data_df = assembler.transform(data_df)
# 训练K-Means模型
kmeans = KMeans().setK(3).setMaxIter(1)  # 设置簇的数量
model = kmeans.fit(data_df)

# 获取聚类中心
centers = model.clusterCenters()
print("K-Means聚类中心:")
for center in centers:
    print(center)

24/04/27 22:15:30 WARN TaskSetManager: Stage 0 contains a task of very large size (3287 KiB). The maximum recommended task size is 1000 KiB.
24/04/27 22:15:44 WARN TaskSetManager: Stage 3 contains a task of very large size (3287 KiB). The maximum recommended task size is 1000 KiB.
24/04/27 22:15:49 WARN TaskSetManager: Stage 4 contains a task of very large size (3287 KiB). The maximum recommended task size is 1000 KiB.
24/04/27 22:15:49 WARN TaskSetManager: Stage 5 contains a task of very large size (3287 KiB). The maximum recommended task size is 1000 KiB.
24/04/27 22:15:50 WARN TaskSetManager: Stage 6 contains a task of very large size (3287 KiB). The maximum recommended task size is 1000 KiB.
24/04/27 22:15:50 WARN TaskSetManager: Stage 7 contains a task of very large size (3287 KiB). The maximum recommended task size is 1000 KiB.
24/04/27 22:15:51 WARN TaskSetManager: Stage 8 contains a task of very large size (3287 KiB). The maximum recommended task size is 1000 KiB.
24/04/27 22:1

K-Means聚类中心:
[0.99968083 0.99963608 0.99966761]
[0.30914527 0.22881123 0.2925668 ]
[0.76062251 0.71260888 0.75430361]


In [7]:
from pyspark import SparkConf, SparkContext
import os
import math

def distance(p, q):
    dist = math.sqrt(sum((a - b) ** 2 for a, b in zip(p, q)))
    return dist

def closest_point(q, candidates):
    best_index = 0
    closest = float('inf')
    for i, candidate in enumerate(candidates):
        temp_dist = distance(q, candidate)
        if temp_dist < closest:
            closest = temp_dist
            best_index = i
    return candidates[best_index]

def add_vec(v1, v2):
    new_vector = [a + b for a, b in zip(v1, v2)]
    return new_vector

def average(cluster):
    num_vectors = len(cluster)
    out = [0, 0]
    for vec in cluster:
        out = add_vec(out, vec)
    ret = [x // num_vectors for x in out]
    return ret
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.ml.linalg import Vectors


In [11]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame(image, ["features"])
print((df.count(),len(df.columns)))
print(center.shape)

closest_center_udf = udf(lambda q: closest_point(q, center), IntegerType())
df_with_cluster = df.withColumn("cluster_index", closest_center_udf(df["features"]))

# 获取每个数据点的预测 cluster
cluster_indices = df_with_cluster.select("cluster_index").rdd.flatMap(lambda x: x).collect()

# 根据 cluster_indices 和 center 构建 processed_data
processed_data = np.array([center[i] for i in cluster_indices])

# 打印 processed_data
print(processed_data.shape)
# 关闭 Spark 会话
spark.stop()

24/04/27 22:32:50 WARN TaskSetManager: Stage 22 contains a task of very large size (3287 KiB). The maximum recommended task size is 1000 KiB.


(925508, 3)
(3,)


24/04/27 22:32:54 WARN TaskSetManager: Stage 25 contains a task of very large size (3287 KiB). The maximum recommended task size is 1000 KiB.
24/04/27 22:32:56 ERROR PythonRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/yangyongze/projects/image_quantization/spark-3.5.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 1225, in main
    eval_type = read_int(infile)
                ^^^^^^^^^^^^^^^^
  File "/Users/yangyongze/projects/image_quantization/spark-3.5.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePyt

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/var/folders/qm/7qw4zbbj2g700gltsr5vwcqc0000gn/T/ipykernel_50660/3684650497.py", line 11, in <lambda>
  File "/var/folders/qm/7qw4zbbj2g700gltsr5vwcqc0000gn/T/ipykernel_50660/3288247416.py", line 13, in closest_point
  File "/var/folders/qm/7qw4zbbj2g700gltsr5vwcqc0000gn/T/ipykernel_50660/3288247416.py", line 6, in distance
TypeError: 'float' object is not iterable
